# IBM Applied Data Science Capstone - Week 5 Final Project¶

### In this project we aim to find the best cluster to open Bars in Perth, Western Australia. To do:

1. A dataframe is created to store neighborhoods in Perth, Western Australia by using anf scraping data from Wikipedia page.

2. geocoder package is used to extract geographical coordinates of neighborhoods.

3. Foursquare API is used to capture all the vanue data of the neighborhood. 

4. Neighborhood are clustered in three clusters and finally the best cluster that has the highest potential to open a Bar is selected

In [2]:
# library to handle requests
import requests 

# library for data analsysis
import pandas as pd

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# library to handle data in a vectorized manner
import numpy as np 

# library for random number generation
import random 

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
from IPython.display import display_html

!pip install beautifulsoup4
!pip install lxml
!conda install -c conda-forge folium=0.5.0 --yes
!pip install geocoder

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import folium # plotting library
from bs4 import BeautifulSoup # library to parse HTML and XML documents
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                       

In [3]:
data = requests.get('https://en.wikipedia.org/wiki/Category:Suburbs_of_Perth,_Western_Australia').text

In [4]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [5]:
# create a list to store neighborhood data
neighborhoodList = []

In [6]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    #print (row.text) # this can be use to make sure all data is captures from webpage
    neighborhoodList.append(row.text)

In [7]:
# create a new DataFrame from the list
Neighborhood_df = pd.DataFrame({"Neighborhood": neighborhoodList})

Neighborhood_df.head()

,Neighborhood
0,"► Suburbs of Perth, Western Australia by loca..."
1,"► Applecross, Western Australia‎ (6 P)"
2,"► Armadale, Western Australia‎ (8 P)"
3,"► Ascot, Western Australia‎ (4 P)"
4,"► Balcatta, Western Australia‎ (3 P)"


In [8]:
# print the dimensions of the Neighborhood_df dataframe
Neighborhood_df.shape

(72, 1)

In [15]:
# define a function to get coordinates
# use while loop and loop till the last coordinate is captured 
def get_coords(neighborhood):
 
    coords = None

    while(coords is None):
        g = geocoder.arcgis('{}, Perth, Western Australia'.format(neighborhood))
        coords = g.latlng
    return coords

In [16]:
# call function to get the coordinates, store in a new list using list comprehension
coordinates = [ get_coords(neighborhood) for neighborhood in Neighborhood_df["Neighborhood"].tolist() ]

In [19]:
coordinates

[[-31.95264999999995, 115.85742000000005],
 [-31.98150997608039, 115.77148997027962],
 [-31.98136999829736, 115.77148997027962],
 [-31.981639979398643, 115.77151997749297],
 [-32.600690006558175, 115.63866999728543],
 [-31.910930995271624, 115.95403368872726],
 [-32.60080999350213, 115.63886001503045],
 [-31.955928747637557, 115.84490990414741],
 [-31.981639979398643, 115.77151997749297],
 [-32.600690006558175, 115.63866999728543],
 [-31.981639979398643, 115.77151997749297],
 [-32.06515996580252, 115.86220508714308],
 [-31.980959997503533, 115.77148997027962],
 [-32.08527999095865, 115.91210997473081],
 [-31.98136999829736, 115.77148997027962],
 [-31.983529999999973, 115.78008000000011],
 [-32.60092000598131, 115.63906998170495],
 [-31.981237648046218, 115.77158233885262],
 [-31.98586639487866, 115.82351003812964],
 [-31.907439745713823, 115.98009971834225],
 [-31.948189999999954, 115.84240000000011],
 [-32.26240413099998, 115.76977538000006],
 [-31.86152665324114, 115.89585983616226],

In [20]:
# inserting the coordinates into the Neighborhood_df by storing coordinates into coordinates_df
coordinates_df = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])
Neighborhood_df['Latitude'] = coordinates_df['Latitude']
Neighborhood_df['Longitude'] = coordinates_df['Longitude']

In [21]:
# print and see the neighborhoods and the coordinates
print(Neighborhood_df.shape)
Neighborhood_df

(72, 3)


,Neighborhood,Latitude,Longitude
0,"► Suburbs of Perth, Western Australia by loca...",-31.952650,115.857420
1,"► Applecross, Western Australia‎ (6 P)",-31.981510,115.771490
2,"► Armadale, Western Australia‎ (8 P)",-31.981370,115.771490
3,"► Ascot, Western Australia‎ (4 P)",-31.981640,115.771520
4,"► Balcatta, Western Australia‎ (3 P)",-32.600690,115.638670
5,"► Bassendean, Western Australia‎ (6 P)",-31.910931,115.954034
6,"► Bayswater, Western Australia‎ (5 P)",-32.600810,115.638860
7,"► Bellevue, Western Australia‎ (3 P)",-31.955929,115.844910
8,"► Belmont, Western Australia‎ (4 P)",-31.981640,115.771520
9,"► Bickley, Western Australia‎ (3 P)",-32.600690,115.638670


In [22]:
# obtain Perth, Western_Australia coordinates
address = 'Perth, Western Australia'

geolocator = Nominatim(user_agent="myGeocoder")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Perth, Western Australia {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Perth, Western Australia -31.9527121, 115.8604796.


In [23]:
# create map of Perth, Western Australia using latitude and longitude values
map_Neighborhood = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(Neighborhood_df['Latitude'], Neighborhood_df['Longitude'], Neighborhood_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    #label = folium.Popup(label, parse_html=True)
    label = folium.Popup(str(label), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
    parse_html=False).add_to(map_Neighborhood)  
    
map_Neighborhood

In [26]:
#define Foursquare Credentials and version
CLIENT_ID=''
CLIENT_SECRET=''
VERSION='20180605'

In [27]:
# Get 100 venues with in 2000 meters 
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(Neighborhood_df['Latitude'], Neighborhood_df['Longitude'], Neighborhood_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [28]:
# convert the venues list into a new DataFrame
ven_df = pd.DataFrame(venues)

# define the column names
ven_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

In [29]:
print(ven_df.shape)
ven_df.head()

(2763, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"► Suburbs of Perth, Western Australia by loca...",-31.95265,115.85742,Varnish on King,-31.950952,115.855536,Whisky Bar
1,"► Suburbs of Perth, Western Australia by loca...",-31.95265,115.85742,Max + Sons,-31.951645,115.858371,Coffee Shop
2,"► Suburbs of Perth, Western Australia by loca...",-31.95265,115.85742,Ribs & Burgers,-31.951655,115.858375,Burger Joint
3,"► Suburbs of Perth, Western Australia by loca...",-31.95265,115.85742,Toastface Grillah,-31.952441,115.860964,Sandwich Place
4,"► Suburbs of Perth, Western Australia by loca...",-31.95265,115.85742,Twilight Hawkers Markets,-31.953079,115.859452,Food Truck


In [30]:
# grouping by neighborhood to find out number of venues retuend by each neighborhood
ven_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"► Applecross, Western Australia‎ (6 P)",51,51,51,51,51,51
"► Armadale, Western Australia‎ (8 P)",50,50,50,50,50,50
"► Ascot, Western Australia‎ (4 P)",52,52,52,52,52,52
"► Balcatta, Western Australia‎ (3 P)",10,10,10,10,10,10
"► Bassendean, Western Australia‎ (6 P)",25,25,25,25,25,25
"► Bayswater, Western Australia‎ (5 P)",10,10,10,10,10,10
"► Bellevue, Western Australia‎ (3 P)",100,100,100,100,100,100
"► Belmont, Western Australia‎ (4 P)",52,52,52,52,52,52
"► Bickley, Western Australia‎ (3 P)",10,10,10,10,10,10


In [31]:
print('Unique categories: {}.'.format(len(ven_df['VenueCategory'].unique())))

# print out the list of categories
ven_df['VenueCategory'].unique()[:50]

Unique categories: 177.


array(['Whisky Bar', 'Coffee Shop', 'Burger Joint', 'Sandwich Place',
       'Food Truck', 'Vietnamese Restaurant', 'Ice Cream Shop',
       'Pizza Place', 'Bar', 'Beer Bar', 'Hotel', 'Bistro', 'BBQ Joint',
       'Tapas Restaurant', 'Theater', 'Australian Restaurant',
       'Korean Restaurant', 'Dessert Shop', 'Speakeasy', 'Cocktail Bar',
       'Harbor / Marina', 'Portuguese Restaurant', 'Restaurant',
       'Music Venue', 'Gastropub', 'Sushi Restaurant', 'Steakhouse',
       'Middle Eastern Restaurant', 'Plaza',
       'Vegetarian / Vegan Restaurant', 'Japanese Restaurant',
       'Asian Restaurant', 'Stadium', 'Electronics Store',
       'Department Store', 'Food Court', 'Café', 'Thai Restaurant',
       'Scenic Lookout', 'Bridge', 'Dive Bar', 'Mexican Restaurant',
       'Chinese Restaurant', 'Gym', 'Shopping Mall', 'Meze Restaurant',
       'Jazz Club', 'Pastry Shop', 'Pedestrian Plaza', 'Garden'],
      dtype=object)

In [32]:
# To make sure that the scope of project is correct here e check to see if the venue category contain "Bars"
"Bar" in ven_df['VenueCategory'].unique()

True

In [33]:
# Here we want ot create a new dataframe that stores neighborhood and their venue category
# First,creating new dataframe for venues category

Neighborhood_Ven_df = pd.get_dummies(ven_df[['VenueCategory']], prefix="", prefix_sep="")

Neighborhood_Ven_df.head(97)

,Airport,Animal Shelter,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Vineyard,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
# Second,append neighborhood column back to Neighborhood_Ven_df
Neighborhood_Ven_df['Neighborhood'] = ven_df['Neighborhood'] 

print(Neighborhood_Ven_df.shape)
Neighborhood_Ven_df.head(97)

(2763, 178)


,Airport,Animal Shelter,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Video Store,Vietnamese Restaurant,Vineyard,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,"► Suburbs of Perth, Western Australia by loca..."
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"► Suburbs of Perth, Western Australia by loca..."
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"► Suburbs of Perth, Western Australia by loca..."
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"► Suburbs of Perth, Western Australia by loca..."
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"► Suburbs of Perth, Western Australia by loca..."
5,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,"► Suburbs of Perth, Western Australia by loca..."
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"► Suburbs of Perth, Western Australia by loca..."
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"► Suburbs of Perth, Western Australia by loca..."
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"► Suburbs of Perth, Western Australia by loca..."
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"► Suburbs of Perth, Western Australia by loca..."


In [35]:
# Third, if you check neighborhood column is the last column, hence we move it to the first column

fixed_columns = [Neighborhood_Ven_df.columns[-1]] + list(Neighborhood_Ven_df.columns[:-1])
Neighborhood_Ven_df = Neighborhood_Ven_df[fixed_columns]

print(Neighborhood_Ven_df.shape)
Neighborhood_Ven_df.head(97)

(2763, 178)


,Neighborhood,Airport,Animal Shelter,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,...,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Vineyard,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,"► Suburbs of Perth, Western Australia by loca...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,"► Suburbs of Perth, Western Australia by loca...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"► Suburbs of Perth, Western Australia by loca...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"► Suburbs of Perth, Western Australia by loca...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"► Suburbs of Perth, Western Australia by loca...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"► Suburbs of Perth, Western Australia by loca...",0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,"► Suburbs of Perth, Western Australia by loca...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"► Suburbs of Perth, Western Australia by loca...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"► Suburbs of Perth, Western Australia by loca...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"► Suburbs of Perth, Western Australia by loca...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# Here we group Neighborhood_Ven_df by neighberhood and put it in new dataframe
Neighborhoods_grouped = Neighborhood_Ven_df.groupby(["Neighborhood"]).mean().reset_index()

print(Neighborhoods_grouped.shape)
Neighborhoods_grouped

(71, 178)


,Neighborhood,Airport,Animal Shelter,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,...,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Vineyard,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,"► Applecross, Western Australia‎ (6 P)",0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.039216,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.00,0.019608,0.00,0.019608,0.000,0.000
1,"► Armadale, Western Australia‎ (8 P)",0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.00,0.020000,0.00,0.020000,0.000,0.000
2,"► Ascot, Western Australia‎ (4 P)",0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.00,0.019231,0.00,0.019231,0.000,0.000
3,"► Balcatta, Western Australia‎ (3 P)",0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.10,0.000000,0.000,0.000
4,"► Bassendean, Western Australia‎ (6 P)",0.04,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.000,0.000
5,"► Bayswater, Western Australia‎ (5 P)",0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.10,0.000000,0.000,0.000
6,"► Bellevue, Western Australia‎ (3 P)",0.00,0.0,0.000000,0.010000,0.030000,0.000000,0.020000,0.010000,0.01,...,0.020000,0.000000,0.020000,0.0,0.01,0.020000,0.00,0.000000,0.000,0.000
7,"► Belmont, Western Australia‎ (4 P)",0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.00,0.019231,0.00,0.019231,0.000,0.000
8,"► Bickley, Western Australia‎ (3 P)",0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.10,0.000000,0.000,0.000
9,"► Bicton, Western Australia‎ (4 P)",0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.00,0.019231,0.00,0.019231,0.000,0.000


In [37]:
# To go to final step, clustring, we need to create a new dataframe only for Bar
Bar_df = Neighborhoods_grouped[["Neighborhood","Bar"]]
Bar_df.head()

,Neighborhood,Bar
0,"► Applecross, Western Australia‎ (6 P)",0.039216
1,"► Armadale, Western Australia‎ (8 P)",0.040000
2,"► Ascot, Western Australia‎ (4 P)",0.038462
3,"► Balcatta, Western Australia‎ (3 P)",0.000000
4,"► Bassendean, Western Australia‎ (6 P)",0.000000


In [38]:
from sklearn.cluster import KMeans

In [39]:
import sklearn.cluster.k_means_ as kmean
kmeans = kmean.KMeans()

In [40]:
# set number of clusters
Noclusters = 3

clustering_df = Bar_df.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=Noclusters, random_state=0).fit(clustering_df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 0, 0, 0, 1, 2, 0, 2], dtype=int32)

In [41]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Cluster_df = Bar_df.copy()

# add clustering labels
Cluster_df["Cluster Labels"] = kmeans.labels_

# merge bar_grouped with bar_data to add latitude/longitude for each neighborhood
Cluster_df = Cluster_df.join(Neighborhood_df.set_index("Neighborhood"), on="Neighborhood")

print(Cluster_df.shape)
Cluster_df.head(50)

(71, 5)


,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
0,"► Applecross, Western Australia‎ (6 P)",0.039216,2,-31.981510,115.771490
1,"► Armadale, Western Australia‎ (8 P)",0.040000,2,-31.981370,115.771490
2,"► Ascot, Western Australia‎ (4 P)",0.038462,2,-31.981640,115.771520
3,"► Balcatta, Western Australia‎ (3 P)",0.000000,0,-32.600690,115.638670
4,"► Bassendean, Western Australia‎ (6 P)",0.000000,0,-31.910931,115.954034
5,"► Bayswater, Western Australia‎ (5 P)",0.000000,0,-32.600810,115.638860
6,"► Bellevue, Western Australia‎ (3 P)",0.050000,1,-31.955929,115.844910
7,"► Belmont, Western Australia‎ (4 P)",0.038462,2,-31.981640,115.771520
8,"► Bickley, Western Australia‎ (3 P)",0.000000,0,-32.600690,115.638670
9,"► Bicton, Western Australia‎ (4 P)",0.038462,2,-31.981640,115.771520


In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(Noclusters)
ys = [i+x+(i*x)**2 for i in range(Noclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Cluster_df['Latitude'], Cluster_df['Longitude'], Cluster_df['Neighborhood'], Cluster_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [43]:
Cluster_df.sort_values(["Cluster Labels"], inplace=True)
print(Cluster_df.shape)
Cluster_df.head(50)

(71, 5)


,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
35,"► Kenwick, Western Australia‎ (3 P)",0.000000,0,-32.600690,115.638670
27,"► Guildford, Western Australia‎ (10 P)",0.000000,0,-31.907440,115.980100
28,"► Hamilton Hill, Western Australia‎ (5 P)",0.000000,0,-32.121889,115.776523
29,"► Hazelmere, Western Australia‎ (2 P)",0.000000,0,-31.903132,115.992268
31,"► Innaloo, Western Australia‎ (3 P)",0.000000,0,-32.600690,115.638670
32,"► Kalamunda, Western Australia‎ (1 C, 9 P)",0.000000,0,-31.960900,116.031684
33,"► Karrakatta, Western Australia‎ (1 C, 4 P)",0.000000,0,-31.959320,115.791770
39,"► Maylands, Western Australia‎ (9 P)",0.000000,0,-32.601090,115.639200
40,"► Middle Swan, Western Australia‎ (7 P)",0.000000,0,-31.849170,116.014980
41,"► Midland, Western Australia‎ (15 P)",0.000000,0,-32.601640,115.639420


In [44]:
#Cluster 0
Cluster_df.loc[Cluster_df['Cluster Labels'] == 0]

,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
35,"► Kenwick, Western Australia‎ (3 P)",0.000000,0,-32.600690,115.638670
27,"► Guildford, Western Australia‎ (10 P)",0.000000,0,-31.907440,115.980100
28,"► Hamilton Hill, Western Australia‎ (5 P)",0.000000,0,-32.121889,115.776523
29,"► Hazelmere, Western Australia‎ (2 P)",0.000000,0,-31.903132,115.992268
31,"► Innaloo, Western Australia‎ (3 P)",0.000000,0,-32.600690,115.638670
32,"► Kalamunda, Western Australia‎ (1 C, 9 P)",0.000000,0,-31.960900,116.031684
33,"► Karrakatta, Western Australia‎ (1 C, 4 P)",0.000000,0,-31.959320,115.791770
39,"► Maylands, Western Australia‎ (9 P)",0.000000,0,-32.601090,115.639200
40,"► Middle Swan, Western Australia‎ (7 P)",0.000000,0,-31.849170,116.014980
41,"► Midland, Western Australia‎ (15 P)",0.000000,0,-32.601640,115.639420


In [45]:
#Cluster 1
Cluster_df.loc[Cluster_df['Cluster Labels'] == 1]

,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
49,"► Murdoch, Western Australia‎ (1 C, 8 P)",0.050000,1,-31.948190,115.842400
11,"► Burswood, Western Australia‎ (14 P)",0.043478,1,-31.980960,115.771490
59,"► Rockingham, Western Australia‎ (1 C, 9 P)",0.048780,1,-32.282250,115.739001
61,"► South Perth, Western Australia‎ (13 P)",0.050000,1,-31.948190,115.842400
6,"► Bellevue, Western Australia‎ (3 P)",0.050000,1,-31.955929,115.844910
53,"► North Perth, Western Australia‎ (2 C, 2 P)",0.050000,1,-31.948190,115.842400
66,"► West Perth, Western Australia‎ (2 C, 10 P)",0.050000,1,-31.946288,115.845473
19,"► East Perth, Western Australia‎ (2 C, 26 P)",0.050000,1,-31.948190,115.842400
14,"► Claremont, Western Australia‎ (2 C, 15 P)",0.045455,1,-31.983530,115.780080


In [46]:
#Cluster 2
Cluster_df.loc[Cluster_df['Cluster Labels'] == 2]

,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
60,"► South Fremantle, Western Australia‎ (3 P)",0.027778,2,-32.067710,115.753010
63,"► Suburbs of Perth, Western Australia by loca...",0.040000,2,-31.952650,115.857420
64,"► Swanbourne, Western Australia‎ (4 P)",0.038462,2,-31.981640,115.771520
65,"► The Lakes, Western Australia‎ (2 P)",0.033333,2,-31.754460,115.743660
58,"► Redcliffe, Western Australia‎ (2 P)",0.037736,2,-31.981810,115.771510
2,"► Ascot, Western Australia‎ (4 P)",0.038462,2,-31.981640,115.771520
1,"► Armadale, Western Australia‎ (8 P)",0.040000,2,-31.981370,115.771490
23,"► Floreat, Western Australia‎ (6 P)",0.039216,2,-31.981510,115.771490
9,"► Bicton, Western Australia‎ (4 P)",0.038462,2,-31.981640,115.771520
52,"► North Fremantle, Western Australia‎ (1 C, 1 P)",0.029412,2,-32.032490,115.747050



## Conclusion:

Based on the summery page above, most of the Bars are located in Clusters one and two. THe frequency of Bar locations are more in cluster two but cluster 1 has higher number particularly in East Perth and West Perth with aboe 0.5.

Compare to Custer two with moderate number, cluster 0 have very low number to no Bars like as Bayswater, Balcatta and so on.

This means there are high competition in East Perth and West Perth (Cluster 1). Neighborhood in cluster 0 are starving of having Bars. Also, the data illustrates that most of bars are concentrated in east and west areas and not suburb area. Therefore, the project recommends construction developers rely on these findings and invest of suburb areas more. Although those investors who are looking into more competition can also open bars in neighborhood in cluster 2.